In [2]:
# Import beautiful soup
from bs4 import BeautifulSoup
# Import requests
import requests

In [3]:
def get_soup(url):
    # Fetch the page that we're going to parse
    page = requests.get(url)
    # Parse the page with beautiful soup
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup

In [4]:
def get_article_title(soup):
    title = soup.find('h1').text
    return title

In [5]:
def get_article_author(soup):
    author = soup.find('h2', class_='pw-author-name').text
    return author

In [6]:
def get_article_date(soup):
    date = soup.find('p', class_='pw-published-date').text
    return date

In [7]:
# Get the reading time of the article
def get_article_reading_time(soup):
    content = soup.find('div', class_='pw-reading-time').text
    return content

In [46]:
# Breaks the medium article into sections and sub sections based on headers
def get_article_content(soup):
    # Get the content of the article
    content = soup.findAll(class_='pw-post-body-paragraph')
    content = [x for x in content]
    heading_tag = 'h1'
    headings = soup.findAll(heading_tag)
    # Fix for articles with only one main heading
    if(len(headings) < 2):
        heading_tag = 'h2'
        headings = soup.findAll(heading_tag)
    sections = {}
    for heading in headings:
        section = {}
        key = '_content_'
        inner_section = []
        temp = heading
        while(True):
            check = temp.next_sibling
            if check is None:
                break
            if check.name == heading_tag:
                break
            if check.name == 'h2':
                if len(inner_section) > 0:
                    section[key] = ' '.join(inner_section)
                key = check.text
                inner_section = []
            temp = check
            if check.name == 'p':
                inner_section.append(check.text)
        if(len(inner_section) > 0):
            section[key] = ' '.join(inner_section)
        if len(section) > 0:
            sections[heading.text] = section
    return sections

In [12]:
import openai
openai.api_key = 'YOUR_API_KEY'

In [38]:
from IPython.display import display, Markdown

def fetch_summary(sections):
    final_summary = ""
    article_length = 0
    for section_count, section in enumerate(sections):
        display(Markdown(f"## {section}"))
        temp_prompt = ""
        for subsection_count, key in enumerate(sections[section]):
            prompt = sections[section][key]
            if(key != '_content_' and len(prompt.split(" ")) > 100):
                display(Markdown(f"### {key}"))
            temp_prompt += prompt
            if(len(temp_prompt.split(" ")) < 100 and section_count != len(sections) - 1 and subsection_count != len(sections[section]) - 1):
                continue
            article_length += len(temp_prompt.split(" "))
            response = openai.Completion.create(
                model="text-davinci-003",
                prompt=f"Summarize the following text. Do not leave any sentences incomplete. If you reach the maximum length, remove the incomplete sentence: \n\n{temp_prompt}",
                temperature=0.7,
                top_p=1.0,
                max_tokens=150,
                frequency_penalty=0,
                presence_penalty=1
            )
            summary = response['choices'][0]['text']
            if len(summary) > 0:
                display(Markdown(f"{summary}\n"))
                final_summary += summary
            temp_prompt = ""
    return final_summary, article_length

In [11]:
# Article URL
def display_summary(url):
    soup = get_soup(url)
    title = get_article_title(soup)
    author = get_article_author(soup)
    date = get_article_date(soup)
    sections = get_article_content(soup)
    display(Markdown(f"# {title}"))
    display(Markdown(f"Written by **{author}** on **{date}**\n\n"))
    final_summary, article_length = fetch_summary(sections)
    print(f"Length of article: {article_length} words")
    print(f"Length of final summary: {len(final_summary.split(' '))} words")

In [40]:
url = "https://medium.com/sfu-cspmp/the-artistic-potential-of-ai-understanding-dall-e-2-and-stable-diffusion-10a82e6965c0"
display_summary(url)

# The Artistic Potential of AI: Understanding DALL-E 2 and Stable Diffusion

Written by **Machine Minds** on **Feb 11**



## An Overview



AI is transforming the digital world by allowing machines to act and think like humans. Generative AI takes this a step further, introducing the ability to create original visuals, text, sounds, or songs. Models such as DALL-E 2 and Stable Diffusion, developed by OpenAI and LMU Munich respectively, use transformer-based designs and extensive training data to generate images from verbal descriptions, pushing the boundaries of creativity. AI has something to offer for everyone, with endless possibilities in store.


## The Technology Behind DALL-E 2

...

## Try it yourself



OpenAI provides 50 free credits when signing up and 25 extra credits each month for users to create their own images using DALL-E 2. Stable Diffusion is available online for free, as well as a pre-computed model for download, and a Colab Notebook for a code-based implementation. Midjourney is another image generation model with 25 credits given to new Discord accounts, and the Discord Bot can be invited to any server.


## Conclusion


Generative AI is a rapidly expanding field within Artificial Intelligence that has enabled the creation of unique images based on natural language prompts. OpenAI's DALL-E 2 is an advanced model utilizing CLIP and diffusion models, showcasing the potential to revolutionize the digital world. Today, anyone can use this cutting-edge technology to bring their ideas to life.


Length of article: 1799 words
Length of final summary: 455 words


In [74]:
url = "https://medium.com/@johan-jublanc/stop-using-notebooks-or-why-you-should-start-making-data-science-as-a-developer-would-do-5d67c8990df5"
display_summary(url)

# Stop using notebooks (or why you should start making data science like a developer)

Written by **Johan Jublanc** on **Jun 11, 2022**



## What are “notebooks” used for and why they are not effective ?

### Why do we like notebooks as data scientists?



Notebooks are often used at the beginning of data science projects to explore solutions, make a proof of concept and validate technical feasibility. However, they prevent the implementation of good software development practices and require double work when transitioning to production. Additionally, notebooks have four major disadvantages: they are not conducive to the development of complex systems and systematic analysis.


...

### An effort worth making



Data science for developers has a cost of entry, but is worth the effort to set up. In three days, data scientists can be familiar with best practices and apply them in their projects. After some time, they are able to start data science projects in their IDE and use good development practices.


## Conclusion



Stopping using notebooks is difficult as it requires a habit change and learning new tools, but can be done quickly and save time in the short-term while reducing risks. Additionally, this allows data scientists to collaborate more closely with developers, engineers, and other team members, leading to better alignment with standards and best practices, better understanding within the team, and faster results. The ultimate goal should be to integrate experimentation into an MLOps workflow, and getting rid of notebooks is the first step towards this.


Length of article: 1470 words
Length of final summary: 463 words


In [71]:
# Examples of articles

url = "https://medium.com/better-programming/measuring-the-gender-gap-in-animation-using-ai-fac738be4b19"
display_summary(url)

# Measuring the Gender Gap in Animated Films Using Computer Vision

Written by **Mariana Avelino** on **Feb 17**



## The Rise in Female Representation



This analysis of Disney and Pixar films from 1937-2021 reveals that while the ratio of male to female faces has improved in recent years, there is still an overall underrepresentation of female characters. Grouping the films by decade shows a slow but positive trend towards equal gender representation, with a higher standard deviation in recent decades suggesting greater diversity and less formulaic structures and characters.


## The Main-Character Effect

...

## Equal vs Fair Representation



The Bechdel Test has been adopted as an official measure to examine fair female representation in animation films. To pass the test, a film must have two female characters who talk to each other about something besides a man. 25% of films passed the test in the first half of the 20th century, while 78% have passed since the turn of the century. This suggests a correlation between fairness and equality of female representation, with animation being faster in converting to fair representation than equal representation.


## Conclusion



Gender representation in animated films is still unequal, but there are signs of improvement. Female characters often exceed male characters in some films, and it is important not to be too prescriptive about the amount of screen time for either. Films can convey subtle messages that influence how audiences perceive themselves and others, so it is important for genders to be well represented. This could help children see characters they identify with in a variety of roles, inspiring them to reach their full potential.


Length of article: 1364 words
Length of final summary: 497 words


In [69]:
# Examples of articles

url = "https://medium.com/sfu-cspmp/mlflow-a-modern-mlops-tool-for-data-project-collaboration-704ca299d9c3"
display_summary(url)

# MLflow — a modern MLOps tool for data project collaboration

Written by **DeepThinkers** on **Feb 11**



## Introduction



The machine learning lifecycle is a cyclic iterative process with instructions and best practices for developing machine learning algorithms to solve business problems. MLflow is an open-source platform for the complete machine-learning lifecycle that allows users to manage their end-to-end machine-learning workflow. It provides tools to assist with various aspects of the machine learning workflow, making it easier to collaborate and increase velocity in creating projects from experimentation to production.


## MLflow Concepts

...

### The model-deploy pipeline (aka the CD part)



The MLflow Model Registry is used to store qualified models, after which the Data Science team can inspect and evaluate them to decide which model version should be deployed. This decision is encoded in the buildspec.yml of the model-deploy repository and when a commit push is made, an automated CodePipeline workflow will deploy the model to a SageMaker Endpoint. Finally, the endpoint can be tested with sample feature data to get class predictions, thereby completing the CI/CD process with MLflow integration.


## Conclusion



MLflow is an MLOps tool that offers advantages over traditional methods for data project collaboration. It provides a centralized workspace, model registry, deployment integrations and a REST API to manage the entire machine learning lifecycle. It is suitable for starting new projects or improving existing ones.


Length of article: 2427 words
Length of final summary: 1025 words


In [70]:
url = "https://medium.com/sfu-cspmp/squeezenet-the-key-to-unlocking-the-potential-of-edge-computing-c8b224d839ba"
display_summary(url)

# SqueezeNet: The Key to Unlocking the Potential of Edge Computing

Written by **Data Hunters** on **Feb 9**



## AI’s New Frontier: The Rise of SqueezeNet

### What is SqueezeNet?



SqueezeNet is a compact and efficient convolutional neural network algorithm designed to strike a balance between high accuracy and low complexity. It was proposed in 2016 by researchers from DeepScale, UC Berkeley, and Stanford University and consists of Fire modules which combine 1x1 and 3x3 filters to reduce the number of parameters while maintaining high accuracy. SqueezeNet has had numerous advancements and improvements, including SqueezeNet v1.1 and v2.0, which have increased accuracy while decreasing computational expenses. This blog post explores the intricacies of SqueezeNet architecture, its performance and technical nuances, as well as potential applications and possibilities within artificial intelligence.


...

## The Ups and Downs: Navigating Challenges and Limitations



SqueezeNet has the advantage of being efficient and fast, but this comes at a cost of reduced accuracy compared to larger, more complex models. Additionally, its limited number of layers and filters can limit its ability to learn complex features and representations. Furthermore, its scalability is limited and it may not be compatible with other architectures, resulting in limited usefulness in certain applications.


## Wrapping it Up: The Final Verdict



SqueezeNet is a valuable tool for improving efficiency and accuracy in a wide range of real-world applications, from self-driving cars to medical imaging. It is well-suited for these applications due to its ability to perform real-time object detection and classification while consuming minimal computational resources.


Length of article: 2057 words
Length of final summary: 940 words
